# **ÁNH XẠ CÁC ĐIỂM DỮ LIỆU SANG CÁC NODE CỦA OPENSTREETMAP**


In [ ]:
import numpy as np
import pandas as pd
import folium
from scipy.spatial import cKDTree
import itertools

## Load data

In [ ]:
df_node = pd.read_csv('../data/osm_node.csv')
df_node = df_node.rename(columns={'lat':'x', 'lon': 'y'})
df_node

In [ ]:
df_way = pd.read_csv('../data/osm_way.csv')
df_way = df_way.set_index('id')
df_way

In [ ]:
df = pd.read_csv('../data/data_pre.csv')
df

## Xử lý dữ liệu

### Các hàm vẽ giúp trực quan dữ liệu

In [ ]:

def add_point(mapobj, df, colors):
    #Nạp x,y từ dataframe vào list coords
    coords = list(zip(df.x, df.y))
    
    #Hiển thị trên mapobj
    for coord in coords:
        folium.CircleMarker(location = coord,
                            radius = 1.5, 
                            fill = True,
                            fill_opacity = 0.75,
                            color = colors,
                            weight = 0.01).add_to(mapobj)
        
        
def add_lines(mapobj, df, colors):
    coords = list(zip(df.x, df.y))
    folium.PolyLine(coords, color=colors, weight=1, opacity=1).add_to(mapobj)   
       
    

def show_n_route(df, n, type_map):
    '''
    Hàm hiển thị map.
    Tùy chọn:
    - df: dataframe input
    - n: Số lộ trình cần hiển thị
    - type_map: dạng đường (1) và dạng điểm (0)
    '''
    colors = [
        'red',
        'yellow',
        'blue',
        'orange',
        'green',
        'lightgreen',
        'purple',
        'pink']
    #Khởi tạo bản đồ mapobj
    f = folium.Figure(height = 800)
    mapobj = folium.Map([np.median(df.x), np.median(df.y)], zoom_start = 15, tiles='Cartodb dark_matter')
    mapobj.add_to(f)


    #Hiển thị n lộ trình đầu tiên
    index_route = df[df['level_1'] == 0].index
    
    if n == -1 or n> len(index_route)-1:
        n = len(index_route)-1
    
    #Biến đếm màu
    t = 0
    if len(index_route)==1:
        if type_map == 0:
            add_point(mapobj, df, colors[t])
        else:
            add_lines(mapobj, df, colors[t])
        return mapobj
    
    for i in range(n):
        if i == len(index_route):
            df_temp = df.loc[index_route[i]:,['x', 'y']]
            if type_map == 0:
            #df_temp = df.loc[index_route[i]:index_route[i + 1],['x', 'y']]
                add_point(mapobj, df_temp, colors[t])
            else:
                #df_temp = df.loc[index_route[i]:index_route[i + 1] -1,['x', 'y']]
                add_lines(mapobj, df_temp, colors[t])
                add_point(mapobj, df_temp, colors[t])
        else:
            df_temp = df.loc[index_route[i]:index_route[i + 1] -1 ,['x', 'y']]
            if type_map == 0:
                add_point(mapobj, df_temp, colors[t])
            else:
                add_lines(mapobj, df_temp, colors[t])
                add_point(mapobj, df_temp, colors[t])
        t += 1
        if t == 8:
            t = 0
        
    return mapobj

def add_markers(gdf):
    f = folium.Figure(height = 800)
    mapobj = folium.Map([np.median(gdf.x), np.median(gdf.y)], zoom_start = 15, tiles='Cartodb dark_matter')
    mapobj.add_to(f)
    coords = []
    for i, row in gdf.iterrows():
        coords.append([row.x, row.y])
    for coord in coords:
        folium.CircleMarker(location = coord,
                            radius = 2.5, 
                            fill = True,
                            fill_color = '#F50057',
                            fill_opacity = 0.75,
                            color = 'whitesmoke',
                            weight = 0.5).add_to(mapobj)
    return mapobj

In [ ]:
tree = cKDTree(df_node[['x','y']])

In [ ]:
dd, ii = tree.query(df[['x','y']],k=1)
len(ii)

### Hàm kéo các điểm GPS về các node

In [ ]:
# hàm kéo các điểm dữ liệu về node gần nhất, trả về tất cả các id của node theo mỗi xe
def get_nearest(data, df_node):
    '''
    input:
        data: dữ liệu các gps xe
        node: dữ liệu các điểm gps trên đường (dữ liệu chuẩn)
        
    output:
        nearest: dữ liệu gps đã được kéo về các node
        vehicle: dữ liệu các node của mỗi xe
    '''
    nearest = data.copy()
    
    tree = cKDTree(df_node[['x','y']])
    dist, idx = tree.query(data[['x','y']], k=1)
    
    # kéo tất cả các điểm về node gần nhất
    for index in data.index:
        nearest.at[index, 'x'] = df_node.loc[idx[index]]['x']
        nearest.at[index, 'y'] = df_node.loc[idx[index]]['y']
    
    # lấy danh sách các xe
    list_vehicle = list(set(data.vehicle))
    vehicle = {}
    # xét theo mỗi xe
    for veh in list_vehicle:
        list_id_node = []
        # xét theo mỗi điểm của xe
        for point in nearest[nearest.vehicle == veh].index:
            list_id_node.append(df_node[(df_node.x == nearest[nearest.vehicle == veh].loc[point].x) & 
                                        (df_node.y == nearest[nearest.vehicle == veh].loc[point].y)].id.to_list()[0])
        
        
        vehicle[veh] =  [[g for g, _ in itertools.groupby(list_id_node)]]
    
    return nearest, vehicle
%time
df_test, vehicle_node = get_nearest(df, df_node)
df_test

In [ ]:
pd.DataFrame(vehicle_node)

In [ ]:
add_markers(df_test)